In [3]:
import pandas as pd

df = pd.read_csv('data/mm79158.csv', sep=';')


In [4]:
df

,ida_mknmm,register,ts,vrednost,vir
0,75372,A+,2018-01-01 00:00:00,"0,8375",B
1,75372,A+,2018-01-01 00:15:00,"0,5825",B
2,75372,A+,2018-01-01 00:30:00,"0,585",B
3,75372,A+,2018-01-01 00:45:00,"0,5725",B
4,75372,A+,2018-01-01 01:00:00,"0,56",B
...,...,...,...,...,...
121014,421183,A+,2016-12-31 22:45:00,"3,265",B
121015,421183,A+,2016-12-31 23:00:00,"3,36",B
121016,421183,A+,2016-12-31 23:15:00,"3,28",B
121017,421183,A+,2016-12-31 23:30:00,"3,355",B


In [1]:
import pandas as pd

# Load and clean
df = pd.read_csv("data/mm79158.csv", sep=';')
df['ts'] = pd.to_datetime(df['ts'])
df['vrednost'] = df['vrednost'].str.replace(',', '.').astype(float)

# Set timestamp as index
df = df.set_index('ts')

# Resample to hourly by summing up 15-min intervals
df_hourly = df['vrednost'].resample('H').sum()

# Ensure continuous hourly range
full_index = pd.date_range(start=df_hourly.index.min(), end=df_hourly.index.max(), freq='H')
df_hourly = df_hourly.reindex(full_index)

# Interpolate missing values
df_hourly = df_hourly.interpolate(method='time')

# Reset index and rename
df_hourly = df_hourly.reset_index().rename(columns={'index': 'ts', 'vrednost': 'vrednost'})

# Sort ascending
df_hourly = df_hourly.sort_values('ts').reset_index(drop=True)

# Preview result
print(df_hourly.head())


                   ts  vrednost
0 2015-08-12 00:00:00    1.7750
1 2015-08-12 01:00:00    1.6200
2 2015-08-12 02:00:00    1.5725
3 2015-08-12 03:00:00    1.5375
4 2015-08-12 04:00:00    1.8650


C:\Users\janav\AppData\Local\Temp\ipykernel_36396\2105456970.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df['vrednost'].resample('H').sum()
C:\Users\janav\AppData\Local\Temp\ipykernel_36396\2105456970.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=df_hourly.index.min(), end=df_hourly.index.max(), freq='H')


In [2]:
df_hourly[['ts', 'vrednost']].to_csv('data/mm79158_cleaned.csv', index=False)